In [88]:
# Import appropriate modules
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import datetime
import numpy as np
import yfinance as yf
from datetime import datetime
import hvplot
import hvplot.pandas
import matplotlib.pyplot as plt
from vix_functions import garch_fit_and_predict #, retrieve_yahoo_close, retrieve_yahoo_volume #retrieve_yahoo_put_options_volume

# Import of Yahoo Finance data for close prices, volume, and options

In [19]:
#DELETE - FUNCTION IN MODULE
def retrieve_yahoo_close(ticker = 'spy', start_date = '2007-07-01', end_date = '2021-10-03'):
    try:
        # get data based on ticker
        yahoo_data = yf.Ticker(ticker)
        print(f"Processing Close {ticker}")
        # select data using start date and end data and calculate the daily return
        price_df = yahoo_data.history(start=start_date, end=end_date).Close
        price_df.name = ticker
        # if no data retrieved raise exception
        if price_df.shape[0] == 0:
            raise Exception("No Prices.")
        return price_df
    # handle exception
    except Exception as ex:
        print(f"Sorry, Data not available for '{ticker}': Exception is {ex}")

In [20]:
#DELETE - FUNCTION IN MODULE
# Define function to retrieve daily volume data from yahoo using ticker, start date and end date
def retrieve_yahoo_volume(ticker = 'spy', start_date = '2007-07-01', end_date = '2021-10-03'):
    try:
        # get data based on ticker
        yahoo_data = yf.Ticker(ticker)
        print(f"Processing Volume {ticker}")
        # select data using start date and end data and calculate the daily return
        price_df = yahoo_data.history(start=start_date, end=end_date).Volume
        price_df.name = ticker
        # if no data retrieved raise exception
        if price_df.shape[0] == 0:
            raise Exception("No Prices.")
        return price_df
    # handle exception
    except Exception as ex:
        print(f"Sorry, Data not available for '{ticker}': Exception is {ex}")

In [18]:
#DELETE - FUNCTION IN MODULE
# Define function to retrieve put daily volume data from yahoo using ticker, start date and end date
def retrieve_yahoo_put_options_volume(ticker = 'spy', date = '2007-07-01'):
    try:
        # get data based on ticker
        yahoo_data = yf.Ticker(ticker)
        print(f"Processing put volume from {ticker}")
        # select data using start date and end data and calculate the daily return
        opts = yahoo_data.option_chain()
        price_df = opts.puts
        price_df.name = ticker
        price_df = price_df.volume
        # if no data retrieved raise exception
        if price_df.shape[0] == 0:
            raise Exception("No Prices.")
        return price_df
    # handle exception
    except Exception as ex:
        print(f"Sorry, Data not available for '{ticker}': Exception is {ex}")

In [65]:
# Set up the variable related to tickers and tickers data
ticker_data_dict = {}
volume_data_dict = {}
put_volume_data_dict = {}
ticker_list = ["^VIX", "spy", "DX-Y.NYB", "tlt", "ief", "gld", "slv", "CL=F", "USDJPY=X", "VXX", "FXI", "EZU", "EEM", "EFA"]
volume_list = ["^VIX", "spy", "DX-Y.NYB", "tlt", "ief", "gld", "slv", "VXX", "FXI", "EZU", "EEM", "EFA"]
put_volume_list = ["spy", "gld", "VXX", "FXI", "EZU", "EFA"]

# Retrieve Tickers
def retrieve_tickers(ticker_data_dict, ticker_list):
    for ticker in ticker_list:
        ticker_data = retrieve_yahoo_close(ticker)
        ticker_data_dict[ticker] = ticker_data
    return ticker_data_dict

# Retrieve Volume
def retrieve_volume(volume_data_dict, volume_list):
    for ticker in volume_list:        
        volume_data = retrieve_yahoo_volume(ticker)
        volume_data_dict[ticker] = volume_data
    return volume_data_dict

# Retrieve Tickers
def retrieve_put_volume(put_volume_data_dict, put_volume_list):
    for ticker in put_volume_list:
        put_volume_data=retrieve_yahoo_put_options_volume(ticker)
        put_volume_data_dict[ticker] = put_volume_data
    return put_volume_data_dict
        


In [22]:
ticker_data_dict  = retrieve_tickers(ticker_data_dict, ticker_list)
volume_data_dict  = retrieve_volume(volume_data_dict, volume_list)
# put_volume_data_dict = retrieve_put_volume(put_volume_data_dict, put_volume_list)

ticker_data_df     = pd.DataFrame(ticker_data_dict)
volume_data_df     = pd.DataFrame(volume_data_dict)
# put_volume_data_df = pd.DataFrame(put_volume_data_dict)

# ticker_data_df = ticker_data_df.dropna()
# Display the ticker data related data frame
#print(ticker_data_df.head())
#print(volume_data_df.head())
## print(put_volume_data_df.head()) #Option data will be used in a next version of the model

Processing Close ^VIX
Processing Close spy
Processing Close DX-Y.NYB
Processing Close tlt
Processing Close ief
Processing Close gld
Processing Close slv
Processing Close CL=F
Processing Close USDJPY=X
Processing Close VXX
Processing Close FXI
Processing Close EZU
Processing Close EEM
Processing Close EFA
Processing Volume ^VIX
Processing Volume spy
Processing Volume DX-Y.NYB
Processing Volume tlt
Processing Volume ief
Processing Volume gld
Processing Volume slv
Processing Volume VXX
Processing Volume FXI
Processing Volume EZU
Processing Volume EEM
Processing Volume EFA


In [ ]:
# Put Volume data do not have the date - it looks a little complicated to understand per now. 
# Will focus in other variables
put_volume_data_df.tail(20)

## Calculation of returns, and covariances

In [23]:
returns_df=ticker_data_df.pct_change()
returns_all_but_VIX=returns_df.drop(columns=['^VIX'])

In [24]:
cum_returns=(1+returns_df).cumprod()
cum_returns_all_but_vix=(1+returns_all_but_VIX).cumprod()
cum_returns_all_but_vix

,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,VXX,FXI,EZU,EEM,EFA
Date,,,,,,,,,,,,,
2007-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-07-03,1.003624,1.000000,0.993903,0.997034,0.995694,0.995551,1.000000,1.001963,NaN,1.020957,1.006444,1.007968,1.004759
2007-07-04,1.003624,1.000000,0.993903,0.997034,0.995694,0.995551,1.000000,1.003518,NaN,1.020957,1.006444,1.007968,1.004759
2007-07-05,1.002569,1.001843,0.982765,0.991471,0.990772,0.990229,1.010128,1.005890,NaN,1.017375,1.001255,1.012435,0.999756
2007-07-06,1.007840,1.000860,0.978778,0.989122,0.999077,1.007309,1.024195,1.009245,NaN,1.042905,1.012135,1.029784,1.006956
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-27,3.877437,1.147315,2.606016,1.984772,2.516764,1.663489,1.061331,0.906193,0.224331,1.224108,1.278120,1.516358,1.496870
2021-09-28,3.799300,1.152107,2.565423,1.977204,2.492310,1.650779,1.059080,0.907829,0.247740,1.225686,1.247639,1.496776,1.464228
2021-09-29,3.805695,1.159110,2.569874,1.979268,2.481083,1.584843,1.052609,0.912444,0.251356,1.216850,1.241492,1.482832,1.460870


In [25]:
# Are levels correlated? -- YES
ticker_data_df.corr()

,^VIX,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,VXX,FXI,EZU,EEM,EFA
^VIX,1.000000,-0.261907,-0.177535,-0.153038,-0.238696,-0.140994,-0.116533,-0.154325,-0.236938,0.375032,-0.372784,-0.412653,-0.459856,-0.457462
spy,-0.261907,1.000000,0.694249,0.885115,0.866796,0.482224,-0.112768,-0.481942,0.529305,-0.868761,0.743588,0.813923,0.791792,0.924471
DX-Y.NYB,-0.177535,0.694249,1.000000,0.770689,0.754411,0.123592,-0.455048,-0.830481,0.676866,0.274011,0.380284,0.377936,0.239417,0.541566
tlt,-0.153038,0.885115,0.770689,1.000000,0.982691,0.623439,-0.040164,-0.623989,0.398247,-0.356806,0.529986,0.525915,0.571650,0.716657
ief,-0.238696,0.866796,0.754411,0.982691,1.000000,0.680952,0.045964,-0.569718,0.344841,-0.469316,0.493251,0.491168,0.581632,0.706152
gld,-0.140994,0.482224,0.123592,0.623439,0.680952,1.000000,0.714587,-0.023258,-0.360724,-0.539503,0.256068,0.149086,0.545887,0.340552
slv,-0.116533,-0.112768,-0.455048,-0.040164,0.045964,0.714587,1.000000,0.477390,-0.711848,-0.681930,-0.082024,-0.188611,0.238945,-0.117819
CL=F,-0.154325,-0.481942,-0.830481,-0.623989,-0.569718,-0.023258,0.477390,1.000000,-0.514512,-0.340802,-0.226999,-0.154737,-0.013454,-0.270516
USDJPY=X,-0.236938,0.529305,0.676866,0.398247,0.344841,-0.360724,-0.711848,-0.514512,1.000000,0.049850,0.483720,0.627805,0.239002,0.609633
VXX,0.375032,-0.868761,0.274011,-0.356806,-0.469316,-0.539503,-0.681930,-0.340802,0.049850,1.000000,-0.434935,-0.799413,-0.729892,-0.845607


### Observations:
Correlations among levels are particularly high for international/Emergin markets iShares with the VIX: EZU, EFA, EEM, FXI

In [26]:
#Are returns correlated? YES, the most
returns_df.corr()

,^VIX,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,VXX,FXI,EZU,EEM,EFA
^VIX,1.000000,-0.719043,0.093661,0.334191,0.344312,-0.010204,-0.164473,-0.130616,-0.110360,0.886326,-0.532608,-0.641795,-0.609863,-0.661368
spy,-0.719043,1.000000,-0.182516,-0.431964,-0.430172,0.031201,0.220519,0.173980,0.212615,-0.774009,0.743652,0.862075,0.855059,0.905054
DX-Y.NYB,0.093661,-0.182516,1.000000,0.011808,-0.053913,-0.406660,-0.418170,-0.101696,0.051795,0.072226,-0.157699,-0.425560,-0.263495,-0.373556
tlt,0.334191,-0.431964,0.011808,1.000000,0.911065,0.153197,-0.007242,-0.120435,-0.128970,0.280663,-0.331866,-0.415524,-0.363278,-0.407928
ief,0.344312,-0.430172,-0.053913,0.911065,1.000000,0.192801,0.026679,-0.105578,-0.190356,0.311781,-0.360440,-0.401191,-0.376332,-0.396660
gld,-0.010204,0.031201,-0.406660,0.153197,0.192801,1.000000,0.794172,0.071690,-0.037653,-0.066436,0.065263,0.119885,0.141020,0.120403
slv,-0.164473,0.220519,-0.418170,-0.007242,0.026679,0.794172,1.000000,0.124697,0.002264,-0.227686,0.207590,0.288910,0.295210,0.298046
CL=F,-0.130616,0.173980,-0.101696,-0.120435,-0.105578,0.071690,0.124697,1.000000,0.035922,-0.163233,0.128049,0.165415,0.164338,0.171508
USDJPY=X,-0.110360,0.212615,0.051795,-0.128970,-0.190356,-0.037653,0.002264,0.035922,1.000000,-0.013628,0.235700,0.176781,0.233973,0.193523
VXX,0.886326,-0.774009,0.072226,0.280663,0.311781,-0.066436,-0.227686,-0.163233,-0.013628,1.000000,-0.576787,-0.732492,-0.686129,-0.737598


In [27]:
# Is the VIX level correlated with returns? -- Not so much. It's much more correlated with levels,
# or vIX returns correlated with returns

vix_level_and_price_returns_df=pd.concat([ticker_data_df['^VIX'],returns_all_but_VIX], axis=1)
vix_level_and_price_returns_df.corr()

,^VIX,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,VXX,FXI,EZU,EEM,EFA
^VIX,1.000000,-0.136983,0.037840,0.071525,0.080747,0.017460,-0.024428,-0.075762,-0.054239,0.223272,-0.065210,-0.114231,-0.080981,-0.116769
spy,-0.136983,1.000000,-0.182516,-0.431964,-0.430172,0.031201,0.220519,0.173980,0.212615,-0.774009,0.743652,0.862075,0.855059,0.905054
DX-Y.NYB,0.037840,-0.182516,1.000000,0.011808,-0.053913,-0.406660,-0.418170,-0.101696,0.051795,0.072226,-0.157699,-0.425560,-0.263495,-0.373556
tlt,0.071525,-0.431964,0.011808,1.000000,0.911065,0.153197,-0.007242,-0.120435,-0.128970,0.280663,-0.331866,-0.415524,-0.363278,-0.407928
ief,0.080747,-0.430172,-0.053913,0.911065,1.000000,0.192801,0.026679,-0.105578,-0.190356,0.311781,-0.360440,-0.401191,-0.376332,-0.396660
gld,0.017460,0.031201,-0.406660,0.153197,0.192801,1.000000,0.794172,0.071690,-0.037653,-0.066436,0.065263,0.119885,0.141020,0.120403
slv,-0.024428,0.220519,-0.418170,-0.007242,0.026679,0.794172,1.000000,0.124697,0.002264,-0.227686,0.207590,0.288910,0.295210,0.298046
CL=F,-0.075762,0.173980,-0.101696,-0.120435,-0.105578,0.071690,0.124697,1.000000,0.035922,-0.163233,0.128049,0.165415,0.164338,0.171508
USDJPY=X,-0.054239,0.212615,0.051795,-0.128970,-0.190356,-0.037653,0.002264,0.035922,1.000000,-0.013628,0.235700,0.176781,0.233973,0.193523
VXX,0.223272,-0.774009,0.072226,0.280663,0.311781,-0.066436,-0.227686,-0.163233,-0.013628,1.000000,-0.576787,-0.732492,-0.686129,-0.737598


In [ ]:
# VIX returns and volume level is very low
all_returns_and_volume_df=pd.concat([returns_df, volume_data_df], axis=1)
all_returns_and_volume_df.corr()

In [ ]:
# VIX returns and volume change correlation are not significant
all_returns_and_volume_change_df=pd.concat([returns_df, volume_data_df.pct_change()], axis=1)
all_returns_and_volume_df.corr()
#OBS: the second part of the table below are volume. Above are returns.

In [ ]:
# Vix level and volume change -- very low correlation
vix_level_and_volume_change_df=pd.concat([ticker_data_df['^VIX'],volume_data_df.pct_change()], axis=1)
vix_level_and_volume_change_df.corr()

In [28]:
# Vix level and volume level  -- good correlation with some variables
vix_level_and_volume_df=pd.concat([ticker_data_df['^VIX'],volume_data_df], axis=1)
vix_level_and_volume_df.corr()

,^VIX,^VIX,spy,DX-Y.NYB,tlt,ief,gld,slv,VXX,FXI,EZU,EEM,EFA
^VIX,1.000000,NaN,0.729169,-0.008396,-0.015544,0.000935,0.390068,0.135834,0.151463,0.470285,-0.182045,0.414648,0.470047
^VIX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
spy,0.729169,NaN,1.000000,0.002924,-0.046283,-0.217987,0.524100,0.042868,0.130699,0.500993,-0.267414,0.594146,0.376161
DX-Y.NYB,-0.008396,NaN,0.002924,1.000000,0.005505,-0.008652,-0.002241,-0.009548,NaN,-0.006670,0.128083,0.010168,-0.009000
tlt,-0.015544,NaN,-0.046283,0.005505,1.000000,0.557124,0.201424,0.301201,0.518686,0.159572,0.288010,0.177870,0.419974
ief,0.000935,NaN,-0.217987,-0.008652,0.557124,1.000000,-0.031462,0.176520,0.455639,0.134773,0.305947,0.035204,0.332827
gld,0.390068,NaN,0.524100,-0.002241,0.201424,-0.031462,1.000000,0.438859,0.181183,0.278680,-0.181187,0.374205,0.303926
slv,0.135834,NaN,0.042868,-0.009548,0.301201,0.176520,0.438859,1.000000,0.289434,0.011899,-0.051299,0.017222,0.132257
VXX,0.151463,NaN,0.130699,NaN,0.518686,0.455639,0.181183,0.289434,1.000000,0.030863,-0.123935,-0.168446,0.061014
FXI,0.470285,NaN,0.500993,-0.006670,0.159572,0.134773,0.278680,0.011899,0.030863,1.000000,0.128425,0.725271,0.580199


In [29]:
# Results is equivalent than with returns
vix_level_and_cum_price_returns_df=pd.concat([ticker_data_df['^VIX'],cum_returns_all_but_vix], axis=1)
vix_level_and_cum_price_returns_df.corr()

,^VIX,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,VXX,FXI,EZU,EEM,EFA
^VIX,1.000000,-0.262041,-0.178489,-0.153331,-0.239156,-0.141415,-0.116698,-0.154724,-0.236113,0.374093,-0.372870,-0.412586,-0.459930,-0.457451
spy,-0.262041,1.000000,0.694508,0.885289,0.867092,0.481425,-0.113267,-0.480653,0.525528,-0.868670,0.744898,0.814374,0.791834,0.924717
DX-Y.NYB,-0.178489,0.694508,1.000000,0.770235,0.754210,0.122933,-0.454054,-0.828409,0.667626,0.267527,0.380465,0.377631,0.240075,0.541523
tlt,-0.153331,0.885289,0.770235,1.000000,0.982707,0.622268,-0.040824,-0.622110,0.392486,-0.363010,0.530828,0.526749,0.571720,0.717152
ief,-0.239156,0.867092,0.754210,0.982707,1.000000,0.679812,0.045261,-0.567531,0.338265,-0.474758,0.494385,0.492196,0.581857,0.706845
gld,-0.141415,0.481425,0.122933,0.622268,0.679812,1.000000,0.714993,-0.020299,-0.363027,-0.542889,0.257355,0.149678,0.546478,0.340775
slv,-0.116698,-0.113267,-0.454054,-0.040824,0.045261,0.714993,1.000000,0.478520,-0.712142,-0.681076,-0.081079,-0.188393,0.239236,-0.117659
CL=F,-0.154724,-0.480653,-0.828409,-0.622110,-0.567531,-0.020299,0.478520,1.000000,-0.501147,-0.334776,-0.224262,-0.153102,-0.011567,-0.268554
USDJPY=X,-0.236113,0.525528,0.667626,0.392486,0.338265,-0.363027,-0.712142,-0.501147,1.000000,0.052021,0.481712,0.627137,0.236679,0.606907
VXX,0.374093,-0.868670,0.267527,-0.363010,-0.474758,-0.542889,-0.681076,-0.334776,0.052021,1.000000,-0.437148,-0.797584,-0.727571,-0.844179


In [83]:
# Vix level and cuadratic returns -- YES, STRONG CORRELATIONS WITH SEVERAL OF THEM
cuadratic_returns_all_but_VIX=returns_all_but_VIX**2
vix_level_and_cuadratic_price_returns_df=pd.concat([ticker_data_df['^VIX'],cuadratic_returns_all_but_VIX], axis=1)
vix_level_and_cuadratic_price_returns_df.corr()
high_correlation_with_vix_level=vix_level_and_cuadratic_price_returns_df['^VIX'].loc[vix_level_and_cuadratic_price_returns_df['^VIX']>.20]

vix_level_and_cuadratic_price_returns_df.dropna().corr()

,^VIX,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,VXX,FXI,EZU,EEM,EFA
^VIX,1.000000,0.596251,0.464580,0.491590,0.422051,0.326108,0.293847,0.106188,0.348273,0.411378,0.432781,0.538807,0.559748,0.553911
spy,0.596251,1.000000,0.382286,0.450573,0.490611,0.334591,0.406099,0.015337,0.362656,0.635882,0.670491,0.911171,0.892006,0.936540
DX-Y.NYB,0.464580,0.382286,1.000000,0.395936,0.313815,0.220726,0.136854,-0.008298,0.166925,0.192054,0.304164,0.342452,0.419397,0.363877
tlt,0.491590,0.450573,0.395936,1.000000,0.923144,0.171886,0.240586,0.003579,0.533954,0.340343,0.394468,0.374960,0.522731,0.392700
ief,0.422051,0.490611,0.313815,0.923144,1.000000,0.170715,0.273887,-0.001960,0.492276,0.395203,0.436998,0.400601,0.552647,0.421886
gld,0.326108,0.334591,0.220726,0.171886,0.170715,1.000000,0.616827,-0.003618,0.039385,0.100764,0.181008,0.319417,0.262607,0.339436
slv,0.293847,0.406099,0.136854,0.240586,0.273887,0.616827,1.000000,-0.005111,0.122181,0.277932,0.326034,0.404178,0.431355,0.402679
CL=F,0.106188,0.015337,-0.008298,0.003579,-0.001960,-0.003618,-0.005111,1.000000,-0.006690,0.032283,-0.002630,0.006091,0.006082,0.009386
USDJPY=X,0.348273,0.362656,0.166925,0.533954,0.492276,0.039385,0.122181,-0.006690,1.000000,0.275244,0.227799,0.312648,0.321165,0.302474
VXX,0.411378,0.635882,0.192054,0.340343,0.395203,0.100764,0.277932,0.032283,0.275244,1.000000,0.486941,0.640133,0.647078,0.634878


In [34]:
#VIX return and returns squared of securities -- not very much
vix_return_and_cuadratic_price_returns_df=pd.concat([ticker_data_df['^VIX'].pct_change(),cuadratic_returns_all_but_VIX], axis=1)
vix_return_and_cuadratic_price_returns_df.corr()

,^VIX,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,VXX,FXI,EZU,EEM,EFA
^VIX,1.000000,0.124305,0.067106,0.096339,0.119791,0.105155,0.087879,0.030571,0.011232,0.570340,0.046737,0.155752,0.058198,0.133199
spy,0.124305,1.000000,0.226996,0.347405,0.325047,0.222841,0.271806,0.010495,0.099481,0.636786,0.718083,0.854088,0.846192,0.925719
DX-Y.NYB,0.067106,0.226996,1.000000,0.242738,0.349730,0.265178,0.277804,-0.003589,0.099423,0.196318,0.202193,0.336686,0.206804,0.285043
tlt,0.096339,0.347405,0.242738,1.000000,0.770791,0.136597,0.157531,0.004132,0.042219,0.342125,0.177323,0.338374,0.233560,0.300293
ief,0.119791,0.325047,0.349730,0.770791,1.000000,0.204029,0.175213,-0.002056,0.068598,0.397013,0.237593,0.340685,0.262698,0.308300
gld,0.105155,0.222841,0.265178,0.136597,0.204029,1.000000,0.684685,-0.001511,0.036286,0.104407,0.321788,0.254232,0.258329,0.255787
slv,0.087879,0.271806,0.277804,0.157531,0.175213,0.684685,1.000000,-0.001459,0.041175,0.280198,0.224820,0.242372,0.230109,0.257252
CL=F,0.030571,0.010495,-0.003589,0.004132,-0.002056,-0.001511,-0.001459,1.000000,-0.000949,0.032666,-0.000995,0.002848,0.001582,0.004245
USDJPY=X,0.011232,0.099481,0.099423,0.042219,0.068598,0.036286,0.041175,-0.000949,1.000000,0.276258,0.130750,0.094417,0.100300,0.100262
VXX,0.570340,0.636786,0.196318,0.342125,0.397013,0.104407,0.280198,0.032666,0.276258,1.000000,0.488962,0.640928,0.648121,0.635670


## Defining Threshold
Condition is a level where, if we predict thaqt return on the VIX, we garantee a return on a VXX investment

In [20]:
# Plot VIX and VXX/5 levels. The adjustment of the VXX to make it comparable with the VIX
VIX_and_VXX_for_plot=pd.concat([ticker_data_df["^VIX"], ticker_data_df["VXX"]/5],axis=1).dropna()
VIX_and_VXX_for_plot.hvplot()

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [22]:
# Calculate returns for VIX and VXX
VIX_and_VXX=pd.concat([ticker_data_df["^VIX"], ticker_data_df["VXX"]],axis=1).dropna()
returns_vix_and_vxx=VIX_and_VXX.pct_change().dropna()
returns_vix_and_vxx.head()

,^VIX,VXX
Date,,
2018-01-29,0.195164,0.069414
2018-01-30,0.068642,0.032792
2018-01-31,-0.084517,0.003273
2018-02-01,-0.005170,-0.050343
2018-02-02,0.285078,0.128594


In [23]:
# Cases of interest
# When both go up together
vix_and_vxx_positive_df=returns_vix_and_vxx[(returns_vix_and_vxx["^VIX"]*returns_vix_and_vxx['VXX']>0) & 
                     (returns_vix_and_vxx["^VIX"]>0)]

In [24]:
# It is 3% a good threshold? It would if most of the time vxx is also positive when vix is above threshold
results=[]

for threshold_bps in range(150,1000,5):
    threshold=threshold_bps/10000
    returns_vxx_negative_with_vix_above_threshold=returns_vix_and_vxx[(returns_vix_and_vxx["^VIX"]>threshold) &
                                                                  (returns_vix_and_vxx["VXX"]<0)]

                                                                  
    cases_vxx_negative_and_vix_above_threshold=returns_vxx_negative_with_vix_above_threshold.describe().iloc[0,0]
    cases_vix_above_threshold=returns_vix_and_vxx[returns_vix_and_vxx["^VIX"]>threshold].describe().iloc[0,0]
    
    results.append({'Threshold return (%)':threshold*100, 
                     "cases_vxx_negative_and_vix_above_threshold":cases_vxx_negative_and_vix_above_threshold,
                     "cases_vix_above_threshold":cases_vix_above_threshold,
                     "Proportion of fail cases (%)": cases_vxx_negative_and_vix_above_threshold/cases_vix_above_threshold*100
                    })
    
results_df=pd.DataFrame(results)

results_df.head(50)

,Threshold return (%),cases_vxx_negative_and_vix_above_threshold,cases_vix_above_threshold,Proportion of fail cases (%)
0,1.50,38.0,334.0,11.377246
1,1.55,38.0,333.0,11.411411
2,1.60,37.0,330.0,11.212121
3,1.65,36.0,325.0,11.076923
4,1.70,36.0,323.0,11.145511
5,1.75,36.0,320.0,11.250000
6,1.80,32.0,316.0,10.126582
7,1.85,30.0,312.0,9.615385
8,1.90,29.0,311.0,9.324759
9,1.95,27.0,309.0,8.737864


In [25]:
curve=results_df.hvplot(x='Threshold return (%)', 
                  y='Proportion of fail cases (%)',
                  title='Determination of Threshold for VIX return prediction'
                 )
amount_of_cases=results_df[(results_df['Threshold return (%)']>2) &
                           (results_df['Threshold return (%)']<4)].hvplot.bar(x='Threshold return (%)', 
                  y=['cases_vxx_negative_and_vix_above_threshold','cases_vix_above_threshold'],
                  title='Determination of Threshold for VIX return prediction'
                 )
curve+amount_of_cases

:Layout
   .Curve.I :Curve   [Threshold return (%)]   (Proportion of fail cases (%))
   .Bars.I  :Bars   [Threshold return (%),Variable]   (value)

## Google Trends data upload, and introduction to the analysis

In [36]:
# Read the google_trends_df.csv file from the Resources folder into a Pandas DataFrame
df_path = Path("./Resources/google_trends_df.csv")
google_trends_df = pd.read_csv(df_path,index_col= 'Date', parse_dates= True, infer_datetime_format=True)

# Review the DataFrame
google_trends_df.T.hvplot.heatmap(
            title= "Google Trends evolution in time",
            height=500, 
            rot=90, 
            width=700)


:HeatMap   [columns,index]   (value)

In [38]:
# Review the data types associated with the columns
google_trends_df.dtypes

Leading_Indicators      int64
PMI                     int64
CCI                     int64
Jobless_Claims          int64
GDP                     int64
war                     int64
Impeachment             int64
Catastrophe             int64
Natural_disaster        int64
Inflation               int64
unemployment            int64
Market_crash            int64
Covid                   int64
virus                   int64
Pandemic                int64
vaccination             int64
Delta                   int64
Covid_mutation          int64
FED                     int64
Tapering                int64
Liquidity               int64
Banking                 int64
Correction_of_market    int64
coup                    int64
Debt_ceiling            int64
retail_spending         int64
Consumer_spending       int64
Consumer                int64
Earnings                int64
Economic_contraction    int64
Depression              int64
Shock                   int64
Monetary_policy         int64
VIX       

##### HERE INCLUDE A FUNCTION THAT CHANGES ANY STRING from the google trends TO 0

In [39]:
# Drop the VXX data, because it doesn't have enough history
ticker_data_df1= ticker_data_df.drop(columns="VXX")
gtd_tdf = pd.concat([ticker_data_df1.pct_change(),google_trends_df],axis=1).dropna()
gtd_tdf.head()

,^VIX,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,FXI,...,Debt_ceiling,retail_spending,Consumer_spending,Consumer,Earnings,Economic_contraction,Depression,Shock,Monetary_policy,VIX
Date,,,,,,,,,,,,,,,,,,,,,
2007-08-01,0.006378,0.004872,0.001238,-0.002165,-0.002771,0.002128,0.006874,-0.021481,0.004729,-0.019370,...,0.0,0.0,0.0,61.0,46.0,0.0,18.0,25.0,0.0,21.0
2007-10-01,-0.008889,0.011273,0.002316,0.005715,0.001197,0.005305,-0.000659,-0.017389,0.009238,0.029167,...,0.0,0.0,24.0,63.0,69.0,0.0,25.0,15.0,25.0,10.0
2007-11-01,0.252563,-0.023407,0.001438,0.012993,0.008162,-0.008776,-0.024861,-0.011002,-0.008060,-0.046588,...,0.0,0.0,0.0,58.0,40.0,0.0,29.0,30.0,25.0,16.0
2008-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001523,0.000000,...,0.0,0.0,0.0,58.0,58.0,0.0,49.0,14.0,0.0,18.0
2008-02-01,-0.083206,0.016088,0.003591,0.006011,0.003245,-0.022429,-0.011833,-0.030409,0.002438,0.046019,...,0.0,0.0,0.0,56.0,64.0,0.0,39.0,21.0,16.0,6.0


## VIX Returns with lag variables - There are several variables and lags that are correlated.
More correlation exist with the VIX level, though.

In [40]:
# Correlations of VIX return with security returns and google trends
correlations_with_vix_return=gtd_tdf.corr().loc["^VIX"].sort_values(ascending=False)
correlations_with_vix_return

^VIX                    1.000000
ief                     0.527580
tlt                     0.508151
Leading_Indicators      0.237956
CCI                     0.216207
virus                   0.139178
war                     0.132465
Jobless_Claims          0.104248
GDP                     0.098281
DX-Y.NYB                0.083312
gld                     0.075658
VIX                     0.057131
Banking                 0.052083
Correction_of_market    0.044169
Delta                   0.040979
Depression              0.039105
Market_crash            0.036494
Consumer                0.036146
PMI                     0.023457
Pandemic                0.022354
FED                     0.013454
Shock                   0.012379
Inflation              -0.002075
Catastrophe            -0.019197
Impeachment            -0.019684
Liquidity              -0.027039
Consumer_spending      -0.030640
Covid                  -0.032681
retail_spending        -0.037443
Monetary_policy        -0.049563
USDJPY=X  

In [41]:
# Correlations of VIX return with google trends and returns of other tickers
# lagged correlations. gtd_t1df means trends in in t-1, but VIX return in t

ret_and_trends_no_vix=gtd_tdf.drop(columns='^VIX')
ret_and_trends_no_vix_lag1=ret_and_trends_no_vix.shift(-1)
ret_and_trends_no_vix_lag2=ret_and_trends_no_vix.shift(-2)
ret_and_trends_no_vix_lag3=ret_and_trends_no_vix.shift(-3)
ret_and_trends_no_vix_lag4=ret_and_trends_no_vix.shift(-4)
ret_and_trends_no_vix_lag5=ret_and_trends_no_vix.shift(-5)
ret_and_trends_no_vix_lag6=ret_and_trends_no_vix.shift(-6)
ret_and_trends_no_vix_lag7=ret_and_trends_no_vix.shift(-7)
vix=gtd_tdf['^VIX']

vix_ret_trends_lagged_correlation=pd.DataFrame()

vix_ret_trends_1=pd.concat([vix,ret_and_trends_no_vix_lag1],axis=1)
vix_ret_trends_2=pd.concat([vix,ret_and_trends_no_vix_lag2],axis=1)
vix_ret_trends_3=pd.concat([vix,ret_and_trends_no_vix_lag3],axis=1)
vix_ret_trends_4=pd.concat([vix,ret_and_trends_no_vix_lag4],axis=1)
vix_ret_trends_5=pd.concat([vix,ret_and_trends_no_vix_lag5],axis=1)
vix_ret_trends_6=pd.concat([vix,ret_and_trends_no_vix_lag6],axis=1)

vix_ret_trends_lagged_correlation[1]=vix_ret_trends_1.corr()['^VIX']
vix_ret_trends_lagged_correlation[2]=vix_ret_trends_2.corr()['^VIX']
vix_ret_trends_lagged_correlation[3]=vix_ret_trends_3.corr()['^VIX']
vix_ret_trends_lagged_correlation[4]=vix_ret_trends_4.corr()['^VIX']
vix_ret_trends_lagged_correlation[5]=vix_ret_trends_5.corr()['^VIX']
vix_ret_trends_lagged_correlation[6]=vix_ret_trends_6.corr()['^VIX']

vix_ret_trends_lagged_correlation.hvplot.heatmap(height=550)

:HeatMap   [columns,index]   (value)

In [42]:
vix_ret_trends_1.corr()['^VIX'].sort_values(ascending=False)
print(vix_ret_trends_1.corr()['^VIX'].sort_values(ascending=False).head(5))
print(vix_ret_trends_1.corr()['^VIX'].sort_values(ascending=False).tail(5))
print("lag1")

^VIX                  1.000000
Leading_Indicators    0.131688
war                   0.114397
Debt_ceiling          0.112854
CL=F                  0.085831
Name: ^VIX, dtype: float64
retail_spending   -0.086460
Monetary_policy   -0.112374
vaccination       -0.128992
Shock             -0.166594
FED               -0.169568
Name: ^VIX, dtype: float64
lag1


In [205]:
vix_ret_trends_2=pd.concat([vix,ret_and_trends_no_vix_lag2],axis=1)

vix_ret_trends_2.corr()['^VIX'].sort_values(ascending=False)
print("lag2")
print(vix_ret_trends_2.corr()['^VIX'].sort_values(ascending=False).head(5))
print(vix_ret_trends_2.corr()['^VIX'].sort_values(ascending=False).tail(3))

lag2
^VIX               1.000000
Monetary_policy    0.188572
GDP                0.181515
EFA                0.166613
Impeachment        0.164921
Name: ^VIX, dtype: float64
CL=F              -0.114323
retail_spending   -0.127028
vaccination       -0.133738
Name: ^VIX, dtype: float64


In [206]:
vix_ret_trends_3=pd.concat([vix,ret_and_trends_no_vix_lag3],axis=1)

vix_ret_trends_3.corr()['^VIX'].sort_values(ascending=False)
print("lag3")
print(vix_ret_trends_3.corr()['^VIX'].sort_values(ascending=False).head(5))
print(vix_ret_trends_3.corr()['^VIX'].sort_values(ascending=False).tail(3))

lag3
^VIX                    1.000000
Economic_contraction    0.296610
Monetary_policy         0.199377
Jobless_Claims          0.186807
Earnings                0.183834
Name: ^VIX, dtype: float64
vaccination          -0.141649
tlt                  -0.141798
Leading_Indicators   -0.150776
Name: ^VIX, dtype: float64


In [207]:
vix_ret_trends_4=pd.concat([vix,ret_and_trends_no_vix_lag4],axis=1)

vix_ret_trends_4.corr()['^VIX'].sort_values(ascending=False)
print("lag4")
print(vix_ret_trends_4.corr()['^VIX'].sort_values(ascending=False).head(5))
print(vix_ret_trends_4.corr()['^VIX'].sort_values(ascending=False).tail(3))

lag4
^VIX         1.000000
USDJPY=X     0.215979
Inflation    0.161238
DX-Y.NYB     0.150253
virus        0.145938
Name: ^VIX, dtype: float64
retail_spending   -0.143722
Tapering          -0.163118
vaccination       -0.171746
Name: ^VIX, dtype: float64


In [208]:
vix_ret_trends_5=pd.concat([vix,ret_and_trends_no_vix_lag5],axis=1)

vix_ret_trends_5.corr()['^VIX'].sort_values(ascending=False)
print("lag5")
print(vix_ret_trends_5.corr()['^VIX'].sort_values(ascending=False).head(5))
print(vix_ret_trends_5.corr()['^VIX'].sort_values(ascending=False).tail(3))

lag5
^VIX                  1.000000
retail_spending       0.370823
Leading_Indicators    0.235101
FXI                   0.178446
Shock                 0.166594
Name: ^VIX, dtype: float64
CCI          -0.173721
Depression   -0.198784
Tapering     -0.203270
Name: ^VIX, dtype: float64


In [209]:
vix_ret_trends_6=pd.concat([vix,ret_and_trends_no_vix_lag6],axis=1)

vix_ret_trends_6.corr()['^VIX'].sort_values(ascending=False)
print(vix_ret_trends_6.corr()['^VIX'].sort_values(ascending=False).head(5))
print(vix_ret_trends_6.corr()['^VIX'].sort_values(ascending=False).tail(3))

^VIX            1.000000
Pandemic        0.203891
Covid           0.190965
virus           0.190551
unemployment    0.173107
Name: ^VIX, dtype: float64
CL=F           -0.107550
Debt_ceiling   -0.113987
Tapering       -0.124115
Name: ^VIX, dtype: float64


# Correlations VIX level with lag variables
The VIX level looks well correlated with our lag variables, more than the VIX return.

In [212]:
# Lag correlations with vix level  -- VIX LEVEL VERY CORRELATED WITH LAGGED VARIABLES
vix_level=ticker_data_df['^VIX']

vix_level_ret_trends_lagged_correlation=pd.DataFrame()

vix_level_ret_trends_1=pd.concat([vix_level,ret_and_trends_no_vix_lag1],axis=1)
vix_level_ret_trends_2=pd.concat([vix_level,ret_and_trends_no_vix_lag2],axis=1)
vix_level_ret_trends_3=pd.concat([vix_level,ret_and_trends_no_vix_lag3],axis=1)
vix_level_ret_trends_4=pd.concat([vix_level,ret_and_trends_no_vix_lag4],axis=1)
vix_level_ret_trends_5=pd.concat([vix_level,ret_and_trends_no_vix_lag5],axis=1)
vix_level_ret_trends_6=pd.concat([vix_level,ret_and_trends_no_vix_lag6],axis=1)

vix_level_ret_trends_lagged_correlation[1]=vix_level_ret_trends_1.corr()['^VIX']
vix_level_ret_trends_lagged_correlation[2]=vix_level_ret_trends_2.corr()['^VIX']
vix_level_ret_trends_lagged_correlation[3]=vix_level_ret_trends_3.corr()['^VIX']
vix_level_ret_trends_lagged_correlation[4]=vix_level_ret_trends_4.corr()['^VIX']
vix_level_ret_trends_lagged_correlation[5]=vix_level_ret_trends_5.corr()['^VIX']
vix_level_ret_trends_lagged_correlation[6]=vix_level_ret_trends_6.corr()['^VIX']

vix_level_ret_trends_lagged_correlation.hvplot.heatmap(height=550)

:HeatMap   [columns,index]   (value)

# GARCH model fit for Univariate series

In [41]:
from arch import arch_model

Date
2007-07-02         NaN
2007-07-03    0.003623
2007-07-04    0.000000
2007-07-05   -0.001050
2007-07-06    0.005257
                ...   
2021-09-27   -0.002861
2021-09-28   -0.020152
2021-09-29    0.001683
2021-09-30   -0.012222
2021-10-01    0.011884
Freq: B, Name: spy, Length: 3720, dtype: float64

In [72]:
# GARCH(1,1)
basic_gm_model=arch_model(100*returns_df['spy'].dropna(), 
                     p=1, q=1,
                     mean='constant',
                     vol='GARCH'
                    )
basic_gm_model
results_basic_gm=basic_gm_model.fit(update_freq=0)
results_basic_gm.summary()


spy_squared=returns_df['spy']**2
comparison_df=pd.concat([results_basic_gm.conditional_volatility/10, vix,spy_squared.dropna()*200 ], axis=1)
print("Correlations:",comparison_df.corr())
comparison_df.dropna().hvplot()


Optimization terminated successfully    (Exit mode 0)
            Current function value: 4973.709452504372
            Iterations: 11
            Function evaluations: 71
            Gradient evaluations: 11
Correlations:           cond_vol      ^VIX       spy
cond_vol  1.000000  0.191575  0.471100
^VIX      0.191575  1.000000  0.338401
spy       0.471100  0.338401  1.000000


Date
2007-07-03    1.047750
2007-07-04    0.972447
2007-07-05    0.899365
2007-07-06    0.836951
2007-07-09    0.797715
                ...   
2021-09-27    0.885547
2021-09-28    0.834428
2021-09-29    1.141968
2021-09-30    1.051094
2021-10-01    1.100495
Freq: B, Name: cond_vol, Length: 3719, dtype: float64

In [126]:
##DELETE - FUNCTION IN MODULE
def garch_fit_and_prediction(series=returns_df['spy'].dropna(), horizon=1, p=1, q=1, o=1):
    #p=1,q=1, o=1 
    #series=returns_df['spy']
    #horizon=1
    
    """
    This function takes a series of returns, and get back a series of conditional volatility
    modeled using a GJR-GARCH with one shock, and t-student distribution of errors that accept a skew.
    
    In the NN model X is shifted in one lag to be able to predict.
    This GARCH series is the GARCH prediction for the volatility of r series that goes in paralell.
    Once X shift, we want to have (r_{t-1})^2 in one column, and the garch_prediction_t in another. 
    This way we are going to include the ARCH prediction to the model.
    If I think that I need to put e_{t-1}^2 together with garch_prediction_{t-1} I am wrong. please think again.
    """

    series=series.dropna()
    shock_skew_gm_model=arch_model(
                    100*series, 
                    p=p, q=q, o=o,
                    mean='constant',
                    vol='GARCH',
                    dist='skewt'
    )
    #Fit GARCH model and predict
    results_shock_skew_gm=shock_skew_gm_model.fit(update_freq=10)

    conditional_volatility=results_shock_skew_gm.conditional_volatility
    #summary               =results_shock_skew_gm.summary()
    forecast              =results_shock_skew_gm.forecast(horizon=1, reindex=False)

    # Prepare return of the series ready to include to X before shift
    serie_garch_before_shift=conditional_volatility.shift(-1)
    serie_garch_before_shift.iloc[-1,:]=forecast.variance.iloc[-1]

    return serie_garch_before_shift



Iteration:     10,   Func. Count:     95,   Neg. LLF: 4766.396740061081
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4756.558165082275
            Iterations: 17
            Function evaluations: 151
            Gradient evaluations: 17


Date
2007-07-03    0.981228
2007-07-04    0.918090
2007-07-05    0.863944
2007-07-06    0.811338
2007-07-09    0.763721
                ...   
2021-09-27    0.870438
2021-09-28    1.371047
2021-09-29    1.273880
2021-09-30    1.364238
2021-10-01    1.606922
Freq: B, Name: cond_vol, Length: 3719, dtype: float64

In [113]:
# DELETE THIS FUNCTION IS ALREADY CREATED
# GJR-GARCH with one shock, and t-student distribution of errors that accept a skew
shock_skew_gm_model=arch_model(100*returns_df['spy'].dropna(), 
                     p=1, q=1, o=1,
                     mean='constant',
                     vol='GARCH',
                     dist='skewt'
                    )
shock_skew_gm_model
results_shock_skew_gm=shock_skew_gm_model.fit(update_freq=10)
#print(results_shock_skew_gm.summary())

spy_volatility_forecast=results_shock_skew_gm.forecast(horizon=1, reindex=False)
#results_shock_skew_gm.plot()

print(results_shock_skew_gm.conditional_volatility)
print(results_shock_skew_gm.conditional_volatility.shift(-1))
garch_series=results_shock_skew_gm.conditional_volatility.shift(-1)
garch_series.iloc[-1,:]=spy_volatility_forecast.variance.iloc[-1]

print(spy_volatility_forecast.variance.iloc[-1])
print(garch_series)



Iteration:     10,   Func. Count:     95,   Neg. LLF: 4766.396740061081
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4756.558165082275
            Iterations: 17
            Function evaluations: 151
            Gradient evaluations: 17
Date
2007-07-03    1.050837
2007-07-04    0.981228
2007-07-05    0.918090
2007-07-06    0.863944
2007-07-09    0.811338
                ...   
2021-09-27    0.909605
2021-09-28    0.870438
2021-09-29    1.371047
2021-09-30    1.273880
2021-10-01    1.364238
Freq: B, Name: cond_vol, Length: 3719, dtype: float64
Date
2007-07-03    0.981228
2007-07-04    0.918090
2007-07-05    0.863944
2007-07-06    0.811338
2007-07-09    0.763721
                ...   
2021-09-27    0.870438
2021-09-28    1.371047
2021-09-29    1.273880
2021-09-30    1.364238
2021-10-01         NaN
Freq: B, Name: cond_vol, Length: 3719, dtype: float64
h.1    1.606922
Name: 2021-10-01 00:00:00, dtype: float64
Date
2007-07-03    0.981228
2007-07-

In [76]:
garch_series=pd.DataFrame()

# I eliminate the VIX from the Garch list
garch_ticker_list=ticker_list[1:len(ticker_list)]

for ticker in garch_ticker_list:
    print(f"Fitting ticker: {ticker} ")
    garch_series[ticker]=garch_fit_and_predict(returns_df[ticker])
    
garch_series

#it would be good to eliminate the result printing

Fitting ticker: spy 
Iteration:     10,   Func. Count:     95,   Neg. LLF: 4934.59456792485
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4756.558965159824
            Iterations: 16
            Function evaluations: 144
            Gradient evaluations: 16
Fitting ticker: DX-Y.NYB 
Iteration:     10,   Func. Count:     96,   Neg. LLF: 4296.048992998401
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2145.59038760661
            Iterations: 19
            Function evaluations: 176
            Gradient evaluations: 19
Fitting ticker: tlt 
Iteration:     10,   Func. Count:     96,   Neg. LLF: 4630.271894277218
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4616.607751122532
            Iterations: 16
            Function evaluations: 146
            Gradient evaluations: 16
Fitting ticker: ief 
Iteration:     10,   Func. Count:     97,   Neg. LLF: 1831.35682758

,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,VXX,FXI,EZU,EEM,EFA
Date,,,,,,,,,,,,,
2007-07-03,0.981216,0.280286,0.676665,0.367390,0.716682,1.181529,1.347005,0.621592,NaN,2.166241,1.303878,1.839534,1.032929
2007-07-04,0.918076,0.276217,0.660774,0.356563,0.704427,1.162497,1.298491,0.593111,NaN,2.092338,1.246773,1.764199,0.986506
2007-07-05,0.863931,0.274479,0.688249,0.370081,0.698842,1.148336,1.286850,0.568932,NaN,2.024999,1.212386,1.693641,0.966808
2007-07-06,0.811323,0.271168,0.677803,0.363695,0.713995,1.199364,1.304549,0.550591,NaN,2.002364,1.171445,1.641937,0.929669
2007-07-09,0.763705,0.267412,0.667826,0.364867,0.717257,1.179825,1.325986,0.525678,NaN,1.949062,1.123917,1.581860,0.890182
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-27,0.870427,0.295472,0.873649,0.350974,0.854908,1.537968,1.655908,0.418068,4.443565,1.933867,1.143917,1.257078,0.966168
2021-09-28,1.371047,0.302054,0.909681,0.352638,0.857515,1.515919,1.570529,0.406735,7.082735,1.869688,1.475156,1.303559,1.301006
2021-09-29,1.273876,0.320060,0.883466,0.343252,0.845487,1.659408,1.521042,0.417981,6.338020,1.823747,1.423304,1.302802,1.242813


In [68]:
garch_ticker_list=ticker_list[1:len(ticker_list)]
garch_ticker_list

['spy',
 'DX-Y.NYB',
 'tlt',
 'ief',
 'gld',
 'slv',
 'CL=F',
 'USDJPY=X',
 'VXX',
 'FXI',
 'EZU',
 'EEM',
 'EFA']

In [80]:
garch_and_vix_df=pd.concat([garch_series, vix], axis=1)
garch_and_vix_df.corr()

,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,VXX,FXI,EZU,EEM,EFA,^VIX
spy,1.000000,0.561252,0.666157,0.662122,0.646589,0.560961,0.215378,0.492827,0.769043,0.786321,0.888308,0.896159,0.936101,0.353420
DX-Y.NYB,0.561252,1.000000,0.637680,0.746340,0.686143,0.623025,0.171352,0.553756,0.317848,0.646955,0.698184,0.667505,0.694396,0.138627
tlt,0.666157,0.637680,1.000000,0.840398,0.575204,0.539498,0.314918,0.340293,0.451511,0.512526,0.759062,0.620541,0.724649,0.213844
ief,0.662122,0.746340,0.840398,1.000000,0.694682,0.578054,0.200772,0.504904,0.444174,0.678401,0.740819,0.714187,0.744427,0.159126
gld,0.646589,0.686143,0.575204,0.694682,1.000000,0.825253,0.191952,0.624707,0.233180,0.785026,0.696178,0.783463,0.738570,0.159147
slv,0.560961,0.623025,0.539498,0.578054,0.825253,1.000000,0.152854,0.487972,0.115157,0.624726,0.638648,0.660286,0.655320,0.179057
CL=F,0.215378,0.171352,0.314918,0.200772,0.191952,0.152854,1.000000,0.062370,0.114523,0.153507,0.214336,0.201654,0.217042,0.165581
USDJPY=X,0.492827,0.553756,0.340293,0.504904,0.624707,0.487972,0.062370,1.000000,0.542767,0.627132,0.489450,0.609649,0.541436,0.135670
VXX,0.769043,0.317848,0.451511,0.444174,0.233180,0.115157,0.114523,0.542767,1.000000,0.544593,0.661763,0.624859,0.681354,-0.147485
FXI,0.786321,0.646955,0.512526,0.678401,0.785026,0.624726,0.153507,0.627132,0.544593,1.000000,0.764619,0.944552,0.838609,0.227028
